In [ ]:
# download dan install 
nltk.download('stopwords')
nltk.download('punkt_tab')

In [3]:
pip install Sastrawi

In [1]:
#LIHAT DATA
import pandas as pd

df = pd.read_csv("ulasan_avoskin_threading.csv")

df.head()

,Pengguna,Rating,Ulasan
0,nadanabilah_,5,Toner ini cocok untuk kulitku yang bruntusan. ...
1,maidaaa,5,"Ini nyaman di kulit untuk anti aging, aku yg s..."
2,citraardini28,5,Aku udah coba Avoskin Miraculous Refining Tone...
3,nahel,5,Teksturnya kental agak bau aneh tapi hasilnya ...
4,ajeng1717,5,1. Suka banget sama tekstur tonernya yang ring...


In [3]:
#LIHAT DUPLIKASI
df.duplicated(subset=['Ulasan']).sum()
df[df.duplicated(subset=['Ulasan'], keep=False)]
df.drop_duplicates(subset=['Ulasan'], keep='first', inplace=True)

In [5]:
#DATA CLEANING
import re

# Fungsi untuk membersihkan teks
def clean_text(text):
    text = re.sub(r'\d+', '', text)  # Hapus angka    
    text = re.sub(r'@\w+', '', text)  # Hapus mention
    text = re.sub(r'#\w+', '', text)  # Hapus hashtag
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Hapus karakter khusus (hanya menyisakan huruf dan spasi)
    text = re.sub(r'(.)\1{2,}', r'\1', text)  # Menghapus huruf berulang lebih dari 2 kali
    text = text.strip()  # Hapus spasi di awal & akhir
    text = ' '.join([word for word in text.split() if len(word) > 1])  # Hapus kata satu huruf
    return text

# Terapkan pembersihan hanya pada kolom "Ulasan"
df['Ulasan'] = df['Ulasan'].astype(str).apply(clean_text)

# Simpan hasil ke file baru
df.to_csv("ulasan_cleaned.csv", index=False)

# Tampilkan 5 data pertama setelah pembersihan
print(df.head())


        Pengguna  Rating                                             Ulasan
0   nadanabilah_       5  Toner ini cocok untuk kulitku yang bruntusan S...
1        maidaaa       5  Ini nyaman di kulit untuk anti aging aku yg se...
2  citraardini28       5  Aku udah coba Avoskin Miraculous Refining Tone...
3          nahel       5  Teksturnya kental agak bau aneh tapi hasilnya ...
4      ajeng1717       5  Suka banget sama tekstur tonernya yang ringan ...


In [7]:
#CASE FOLDING
# Load data hasil cleaning
df = pd.read_csv("ulasan_cleaned.csv")

# Case Folding: Ubah semua teks menjadi huruf kecil
df['Ulasan'] = df['Ulasan'].str.lower()

# Simpan hasil case folding ke file CSV baru
df.to_csv("ulasan_casefolded.csv", index=False)

# Cek hasilnya (opsional)
print(df.head())


        Pengguna  Rating                                             Ulasan
0   nadanabilah_       5  toner ini cocok untuk kulitku yang bruntusan s...
1        maidaaa       5  ini nyaman di kulit untuk anti aging aku yg se...
2  citraardini28       5  aku udah coba avoskin miraculous refining tone...
3          nahel       5  teksturnya kental agak bau aneh tapi hasilnya ...
4      ajeng1717       5  suka banget sama tekstur tonernya yang ringan ...


In [9]:
#TOKENIZATION
import nltk
from nltk.tokenize import word_tokenize
import pandas as pd

try:
    nltk.data.find('tokenizers/punkt')
    print("Punkt ditemukan!")
except LookupError:
    print("Punkt tidak ditemukan, perlu download ulang.")
    

# Baca file CSV
file_path = "ulasan_casefolded.csv" 
df = pd.read_csv(file_path)

# Lakukan tokenisasi pada kolom "Ulasan"
df["Ulasan"] = df["Ulasan"].astype(str).apply(word_tokenize)

# Tampilkan beberapa hasil pertama
print(df[["Ulasan", "Ulasan"]].head())

# Simpan hasilnya ke CSV jika perlu
df.to_csv("ulasan_tokenized.csv", index=False)
print(df.head())

Punkt ditemukan!
                                              Ulasan  \
0  [toner, ini, cocok, untuk, kulitku, yang, brun...   
1  [ini, nyaman, di, kulit, untuk, anti, aging, a...   
2  [aku, udah, coba, avoskin, miraculous, refinin...   
3  [teksturnya, kental, agak, bau, aneh, tapi, ha...   
4  [suka, banget, sama, tekstur, tonernya, yang, ...   

                                              Ulasan  
0  [toner, ini, cocok, untuk, kulitku, yang, brun...  
1  [ini, nyaman, di, kulit, untuk, anti, aging, a...  
2  [aku, udah, coba, avoskin, miraculous, refinin...  
3  [teksturnya, kental, agak, bau, aneh, tapi, ha...  
4  [suka, banget, sama, tekstur, tonernya, yang, ...  
        Pengguna  Rating                                             Ulasan
0   nadanabilah_       5  [toner, ini, cocok, untuk, kulitku, yang, brun...
1        maidaaa       5  [ini, nyaman, di, kulit, untuk, anti, aging, a...
2  citraardini28       5  [aku, udah, coba, avoskin, miraculous, refinin...
3          n

In [49]:
import pandas as pd
import ast
import re

# Fungsi aman untuk mengonversi string list ke list Python
def safe_eval(s):
    try:
        return ast.literal_eval(s)
    except (ValueError, SyntaxError):
        return s.split()

# Fungsi untuk membaca kamus dari file .txt dengan format kata_typo,perbaikan
def load_kamus_typo(filename):
    kamus = {}
    with open(filename, "r", encoding="utf-8") as file:
        for line in file:
            line = line.strip()
            if line:
                parts = line.split(",")
                if len(parts) == 2:
                    key, value = map(str.strip, parts)
                    kamus[key] = value
    return kamus

# Load kamus typo dari file
kamus_typo = load_kamus_typo("kamus_typo.txt")
print("Kamus Typo Contoh (5):", list(kamus_typo.items())[:5])

# Normalisasi kata berdasarkan kamus typo
def normalisasi_kamus(tokens, kamus):
    return [kamus.get(token, token) for token in tokens]


def normalize_hyphenated_words(tokens, kamus):
    normalized_tokens = []
    
    for token in tokens:
        if "-" in token:  # Cek apakah token mengandung tanda "-"
            if token in kamus:  # Cek apakah ada di kamus secara langsung
                normalized_tokens.append(kamus[token])  # Gunakan kata dari kamus
            else:
                # Pisahkan berdasarkan "-" lalu normalisasi bagian-bagiannya
                parts = token.split("-")
                normalized_parts = [kamus.get(part, part) for part in parts]
                normalized_tokens.append("-".join(normalized_parts))  # Gabungkan kembali
        else:
            normalized_tokens.append(kamus.get(token, token))  # Normalisasi biasa
            
    return normalized_tokens


# Normalisasi pengulangan huruf di akhir kata
def normalize_trailing_repeated_letters(word):
    return re.sub(r"(.)\1+$", r"\1", word)

# Normalisasi pengulangan kata berturut-turut3
def normalize_repeated_words(tokens):
    return [tokens[i] if i == 0 or tokens[i] != tokens[i - 1] else f"{tokens[i]}-{tokens[i]}" for i in range(len(tokens))]

# Hapus token yang terdiri dari 2 huruf, kecuali yang bermakna
KATA_PENTING_2_HURUF = {"di", "ke", "ya", "ku", "mu", "si"}

def hapus_token_pendek(tokens):
    return [token for token in tokens if len(token) > 2 or token in KATA_PENTING_2_HURUF]

#  **Pipeline Normalisasi**
def normalisasi_pipeline(ulasan):
    tokens = safe_eval(ulasan)
    tokens = [normalize_trailing_repeated_letters(token) for token in tokens]
    tokens = normalisasi_kamus(tokens, kamus_typo)
    tokens = normalize_hyphenated_words(tokens, kamus_typo)
    tokens = normalize_repeated_words(tokens)
    tokens = normalisasi_kamus(tokens, kamus_typo)  # Pastikan semua kata sudah dinormalisasi
    tokens = hapus_token_pendek(tokens)
    return tokens

# 🗂 **Load Data dan Terapkan Normalisasi**
df = pd.read_csv("ulasan_tokenized.csv")
df["Ulasan"] = df["Ulasan"].apply(normalisasi_pipeline)
df.to_csv("ulasan_normalized_final.csv", index=False)

print("✅ Preprocessing selesai! Data tersimpan di 'ulasan_normalized_final.csv'.")

Kamus Typo Contoh (5): [('abnormal_words', 'words'), ('ngefeknya', 'ngefek'), ('bahagiabselalu', 'bahagia selalu'), ('moistutizer', 'moisturizer'), ('soso', 'medium')]
✅ Preprocessing selesai! Data tersimpan di 'ulasan_normalized_final.csv'.


In [63]:
#NORMALISASI 3 (minim bug)
import pandas as pd
import re

#Fungsi untuk Memisahkan Kata Berulang (contoh: "poripori" → "pori-pori")
def split_repeated_syllables(word):
    """Mendeteksi pengulangan suku kata dan menambahkan tanda hubung (-)."""
    pattern = r'(\w{3,})\1'  # Cari pola pengulangan minimal 2 huruf
    return re.sub(pattern, r'\1-\1', word)

#Terapkan Normalisasi pada Data
df = pd.read_csv("ulasan_normalized_final.csv")

# Terapkan normalisasi suku kata berulang pada setiap token
df["Ulasan"] = df["Ulasan"].apply(lambda tokens: [split_repeated_syllables(token) for token in eval(tokens)])

#Simpan Hasil ke File Baru
df.to_csv("ulasan_normalized_final_v2.csv", index=False)

print("✅ Normalisasi pengulangan suku kata selesai! Data disimpan di 'ulasan_normalized_final_v2.csv'")

✅ Normalisasi pengulangan suku kata selesai! Data disimpan di 'ulasan_normalized_final_v2.csv'


In [65]:
#STOPWORDS
import pandas as pd
import nltk
from nltk.corpus import stopwords

# Stopwords bahasa Indonesia & Inggris
stopwords_indonesia = set(stopwords.words('indonesian'))
stopwords_inggris = set(stopwords.words('english'))

# Stopwords tambahan (kustom)
try:
    with open("stopwords.txt", "r", encoding="utf-8") as f:
        stopwords_kustom = set(f.read().splitlines())  # Simpan dalam set
except FileNotFoundError:
    print("File stopwords.txt tidak ditemukan!")
    stopwords_kustom = set()

# Gabungkan semua stopwords
all_stopwords = stopwords_indonesia | stopwords_inggris | stopwords_kustom

# Baca data yang telah ternormalisasi
df = pd.read_csv("ulasan_normalized_final_v2.csv")

# Fungsi untuk menghapus stopwords
def remove_stopwords(tokens):
    return [word for word in tokens if word not in all_stopwords]

# Terapkan stopwords removal pada kolom "Ulasan"
df["Ulasan"] = df["Ulasan"].apply(lambda x: remove_stopwords(eval(x)))

# Simpan hasil ke file baru
df.to_csv("ulasan_no_stopwords.csv", index=False)
print("ulasan_no_stopwords.csv tersimpan")
print(df.head())

ulasan_no_stopwords.csv tersimpan
        Pengguna  Rating                                             Ulasan
0   nadanabilah_       5  [toner, cocok, kulitku, bruntusan, mengangkat,...
1        maidaaa       5  [nyaman, kulit, anti, aging, sensitive, skin, ...
2  citraardini28       5  [avoskin, miraculous, refining, toner, puas, h...
3          nahel       5  [teksturnya, kental, bau, aneh, hasilnya, menc...
4      ajeng1717       5  [suka, tekstur, tonernya, ringan, cepat, menye...


In [67]:
import pandas as pd
import re

# Load dataset ulasan
df = pd.read_csv("ulasan_no_stopwords.csv")

# Fungsi untuk tokenisasi dan ekstraksi kata unik dari setiap ulasan
def get_unique_words(text):
    words = re.findall(r'\b\w+\b', str(text).lower())  # Ambil kata per kata (huruf dan angka)
    return set(words)  # Ubah ke set agar hanya menyimpan kata unik

# Gabungkan semua kata unik dalam dataset
unique_words = set()
df["Ulasan"].dropna().apply(lambda x: unique_words.update(get_unique_words(x)))

# Simpan kata unik ke DataFrame
df_unique = pd.DataFrame(list(unique_words), columns=["Kata"])

# Simpan kata unik ke CSV
df_unique.to_csv("kata_unik.csv", index=False)
print(f"Kata unik berhasil diekstrak! Total kata unik: {len(df_unique)}")

# Load daftar kata dasar dari vocab.txt
with open("vocab.txt", "r", encoding="utf-8") as f:
    kata_baku = set(f.read().splitlines())  # Simpan dalam set untuk pencarian cepat

# Fungsi untuk mengecek dan mengubah kata dengan imbuhan menjadi kata dasar
def normalize_word(word, vocab):
    if word in vocab:
        return word
    
    # 🔹 Daftar imbuhan awalan dan akhiran
    prefixes = ["ku","di", "me", "men", "meng", "ber", "ter", "nge", "se"]
    suffixes = ["nya", "ku", "kan", "in", "mu"]
    
    # 🔹 Tangani "nge-in" → kata dasar
    if word.startswith("nge") and word.endswith("in"):
        base = word[3:-2]
        if base in vocab:
            return base
    
    # 🔹 Tangani "di-kan", tetapi tetap dianggap kata dikenal jika kata dasarnya ada di vocab
    if word.startswith("di") and word.endswith("kan"):
        base = word[2:-3]  # Hapus "di" dan "kan"
        if base in vocab:
            return word  # Tidak mengubah ke kata dasar, tetap di daftar kata dikenal
    
    # 🔹 Cek kata dengan awalan
    for prefix in prefixes:
        if word.startswith(prefix):
            base = word[len(prefix):]
            if base in vocab:
                return base
    
    # 🔹 Cek kata dengan akhiran
    for suffix in suffixes:
        if word.endswith(suffix):
            base = word[:-len(suffix)]
            if base in vocab:
                return base
    
    return word

# Normalisasi kata berdasarkan vocab
df_unique["Kata Normalized"] = df_unique["Kata"].apply(lambda x: normalize_word(x, kata_baku))

# Simpan kata yang tidak dikenal untuk analisis lebih lanjut
kata_dikenal = df_unique[(df_unique["Kata Normalized"].isin(kata_baku)) | (df_unique["Kata"].str.startswith("di") & df_unique["Kata"].str.endswith("kan"))]
kata_tidak_dikenal = df_unique[~df_unique.index.isin(kata_dikenal.index)]

kata_dikenal.to_csv("kata_dikenal.csv", index=False)
kata_tidak_dikenal.to_csv("kata_tidak_dikenal.csv", index=False)

print(f"Cek selesai! Kata dikenal: {len(kata_dikenal)}, Kata tidak dikenal: {len(kata_tidak_dikenal)}")
print("Kata dikenal disimpan di 'kata_dikenal.csv'")
print("Kata tidak dikenal disimpan di 'kata_tidak_dikenal.csv'")

Kata unik berhasil diekstrak! Total kata unik: 4869
Cek selesai! Kata dikenal: 4184, Kata tidak dikenal: 685
Kata dikenal disimpan di 'kata_dikenal.csv'
Kata tidak dikenal disimpan di 'kata_tidak_dikenal.csv'


In [69]:
kamus_typo = {}

# Load kamus typo dengan pemisah koma
with open("kamus_typo.txt", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if "," in line:  # Pastikan ada pemisah ","
            typo, correct = line.split(",", 1)  # Split hanya pada koma pertama
            kamus_typo[typo.strip()] = correct.strip()
        else:
            print(f"Peringatan: Baris tidak valid di kamus_typo.txt -> '{line}'")

# Load kata tidak dikenal dari CSV
import pandas as pd

df_kata_tidak_dikenal = pd.read_csv("kata_tidak_dikenal.csv", header=None)  # Pastikan tidak ada header
kata_tidak_dikenal = set(df_kata_tidak_dikenal.iloc[:, 0])  # Ambil kolom pertama

# Cari kata yang seharusnya ternormalisasi tetapi masih tidak dikenali
kata_masih_tidak_ternormalisasi = kata_tidak_dikenal.intersection(kamus_typo.keys())

print("Kata yang seharusnya ternormalisasi tetapi masih tidak dikenali:")
print(kata_masih_tidak_ternormalisasi)


Peringatan: Baris tidak valid di kamus_typo.txt -> ''
Peringatan: Baris tidak valid di kamus_typo.txt -> ''
Kata yang seharusnya ternormalisasi tetapi masih tidak dikenali:
{'apapun', 'dipakainya', 'nginkan', 'malem', 'temen', 'nunggu', 'phte', 'bgtbgt', 'guys', 'mid', 'temenku', 'barengan', 'hehe', 'bhap', 'nyesel', 'hua', 'udah', 'gantian', 'nyoba', 'bnr', 'so', 'clekit', 'bling', 'dilengkapi', 'grenjel', 'diambilnya', 'saidhada', 'breng', 'lindungi'}


In [59]:
#STEMMING
import pandas as pd

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import ast

# Buat Stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Baca data setelah stopwords removal
df = pd.read_csv("ulasan_no_stopwords.csv")

# Fungsi stemming
def stemming_tokens(tokens):
    return [stemmer.stem(word) for word in tokens]

# Terapkan stemming pada kolom "Ulasan"
df["Ulasan"] = df["Ulasan"].apply(lambda x: stemming_tokens(ast.literal_eval(x)))

# Simpan hasil ke file baru
df.to_csv("ulasan_stemmed_avoskin.csv", index=False)

print("Stemming selesai, hasil disimpan di ulasan_stemmed_xx.csv")


Stemming selesai, hasil disimpan di ulasan_stemmed_xx.csv


In [61]:
#STOPWORDS
import pandas as pd
import nltk
from nltk.corpus import stopwords

# Stopwords bahasa Indonesia & Inggris
stopwords_indonesia = set(stopwords.words('indonesian'))
stopwords_inggris = set(stopwords.words('english'))

# Stopwords tambahan (kustom)
try:
    with open("stopwords.txt", "r", encoding="utf-8") as f:
        stopwords_kustom = set(f.read().splitlines())  # Simpan dalam set
except FileNotFoundError:
    print("File stopwords.txt tidak ditemukan!")
    stopwords_kustom = set()

# Gabungkan semua stopwords
all_stopwords = stopwords_indonesia | stopwords_inggris | stopwords_kustom

# Baca data yang telah ternormalisasi
df = pd.read_csv("ulasan_documents.csv")

# Fungsi untuk menghapus stopwords
def remove_stopwords(tokens):
    return [word for word in tokens if word not in all_stopwords]

# Terapkan stopwords removal pada kolom "Ulasan"
df["Ulasan"] = df["Ulasan"].apply(lambda x: remove_stopwords(eval(x)))

# Simpan hasil ke file baru
df.to_csv("ulasan_preprocessed_avoskin.csv", index=False)

In [5]:
print(repr(df_stemmed))
print(repr(df_preprocessed))

           Pengguna  Rating                                             Ulasan
0       fonimeilisa       5  ['suka', 'eksfoliasi', 'toner', 'keren', 'prod...
1         latifalya       4  ['bangga', 'habis', 'langsung', 'repurchase', ...
2          vaniashr       5  ['barusan', 'coba', 'produk', 'kemarin', 'beli...
3        luthfiahnn       5  ['bagus', 'banget', 'bikin', 'kulit', 'mulus',...
4          faradits       5  ['jujur', 'bagus', 'tidak tahu', 'avoskin', 'm...
...             ...     ...                                                ...
1995     ndtsabilah       5  ['pakai', 'minggu', 'efek', 'pakai', 'perih', ...
1996     Regyaviony       4  ['toner', 'kandung', 'aktif', 'pakai', 'rada',...
1997       domokyun       3  ['tonernya', 'cair', 'aroma', 'lucu', 'asem-as...
1998      fairuzs13       5  ['eksfoliasi', 'toner', 'semenjak', 'pandemi',...
1999  ratihtresna15       4  ['produk', 'bagus', 'bikin', 'muka', 'lembab',...

[2000 rows x 3 columns]
           Pengguna  Rating

In [3]:
import pandas as pd
import ast
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import SnowballStemmer

# Inisialisasi stemmer Indonesia dan Inggris
factory = StemmerFactory()
ind_stemmer = factory.create_stemmer()
eng_stemmer = SnowballStemmer("english")

# Baca file hasil stopwords removal
df = pd.read_csv("ulasan_no_stopwords.csv")

# Fungsi untuk stem token berdasarkan bahasanya
def stemming_multilang(tokens):
    stemmed_tokens = []
    for word in tokens:
        # Jika karakter semua ASCII dan tidak ada karakter aneh → diasumsikan Bahasa Inggris
        if word.isalpha() and all(ord(c) < 128 for c in word):
            stemmed_word = eng_stemmer.stem(word)
        else:
            stemmed_word = ind_stemmer.stem(word)
        stemmed_tokens.append(stemmed_word)
    return stemmed_tokens

# Terapkan stemming multibahasa
df["Ulasan"] = df["Ulasan"].apply(lambda x: stemming_multilang(ast.literal_eval(x)))

# Simpan hasilnya
df.to_csv("ulasan_stemmed_multilang.csv", index=False)
print("Stemming multibahasa selesai, hasil disimpan di ulasan_stemmed_multilang.csv")


Stemming multibahasa selesai, hasil disimpan di ulasan_stemmed_multilang.csv
